In [18]:
import pandas as pd
import csv
import requests
import numpy as np
import json
import plotly.express as px
import plotly.io as pio


#Download_csv_file

CSV_URL = "https://api.covid19india.org/csv/latest/states.csv"
with requests.Session() as s:
    download = s.get(CSV_URL)


#Read and write to csv

with open('covid19india_states.csv', 'w') as f:
    writer = csv.writer(f)
    for line in download.iter_lines():
        writer.writerow(line.decode('utf-8').split(','))


#Creating Pivot table

states=pd.read_csv('covid19india_states.csv')
table = pd.pivot_table(states, values='Confirmed', index=['State'], columns=['Date'], fill_value=0)
table.to_csv("state_pivot_confirmed.tsv", sep='\t')
pivot_table_confirmed=pd.read_csv("state_pivot_confirmed.tsv", sep="\t")

table = pd.pivot_table(states, values='Tested', index=['State'], columns=['Date'], fill_value=0)
table.to_csv("state_pivot_tested.tsv", sep='\t')
pivot_table_tested=pd.read_csv("state_pivot_tested.tsv", sep="\t")

#Creating two weeks data

last_week_confirmed = pivot_table_confirmed.T.tail(7).T
last_week_confirmed['total_confirmed'] = last_week_confirmed.sum(axis=1)

last_week_tested = pivot_table_tested.T.tail(7).T
last_week_tested['total_tested'] = last_week_tested.sum(axis=1)

first_column_confirmed = pivot_table_confirmed.T.head(2).T
first_column_tested = pivot_table_tested.T.head(2).T

confirmed = first_column_confirmed.join(last_week_confirmed)
tested = first_column_tested.join(last_week_tested)


merged = confirmed.merge(tested, on=['State'])
merged['PositivityRate'] = merged['total_confirmed']/merged['total_tested']


searchfor=['India']
merged2 = merged[~merged.State.str.contains('|'.join(searchfor))]


header=['State','total_confirmed','total_tested','PositivityRate']
merged2.to_csv("state_positivity.tsv", sep='\t',columns=header)


#Creating image
pio.renderers.default = 'browser'

india_states = json.load(open("india_states.geojson", "r"))

state_id_map = {}
for feature in india_states["features"]:
    feature["id"] = feature["properties"]["st_code"]
    state_id_map[feature["properties"]["id"]] = feature["id"]


df = pd.read_csv("state_positivity.tsv",sep='\t')
df["id"] = df["State"].apply(lambda x: state_id_map[x])

fig = px.choropleth(
    df,
    locations="id",
    geojson=india_states,
    color="PositivityRate",
    hover_name="State",
    hover_data=["PositivityRate"],
    title="COVID19 Positivity Rate per week",
    color_continuous_scale=px.colors.diverging.BrBG,
    color_continuous_midpoint=0)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [17]:
df.sort_values(by=['PositivityRate'], inplace=True, ascending=False, ignore_index=True)
df.loc[0:4, ['State', 'PositivityRate']]

,State,PositivityRate
0,Goa,0.182153
1,Maharashtra,0.147155
2,Dadra and Nagar Haveli and Daman and Diu,0.145621
3,Kerala,0.127110
4,Sikkim,0.124437
